In [1]:
import pandas as pd
import requests,json,os
from urllib.parse import quote 

In [2]:
with open('key/sk_open_api_key.txt') as f_:
    sk_key = f_.read()

In [3]:
len(sk_key)

40

In [4]:
# 혼잡도
station_code = 133
dow = 'MON' # 대문자
hh = '08'
url = f"https://apis.openapi.sk.com/puzzle/subway/congestion/stat/train/stations/{station_code}?dow={dow}&hh={hh}"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json",
    "appkey": sk_key
}

response = requests.get(url, headers=headers).json()


In [5]:
response.keys()

dict_keys(['status', 'contents'])

In [6]:
response['contents'].keys()

dict_keys(['subwayLine', 'stationName', 'stationCode', 'stat', 'statStartDate', 'statEndDate'])

In [8]:
len(response['contents']['stat'])

56

In [40]:
response['contents']['stat'][0]['startStationName']

'소요산역'

In [46]:
response['contents']['stationName']

'서울역'

In [146]:
type(response['contents']['stat'][55]['data'][0])

dict

In [12]:
temp_list = []
temp_list2 = []
for stat in response['contents']['stat']:
    if stat['updnLine'] == 1:
        temp_list.append(pd.DataFrame(stat['data']))
    elif stat['updnLine'] == 0:
        temp_list2.append(pd.DataFrame(stat['data']))
# temp_list = temp_list[0]
# temp_list
# temp_list2 = temp_list2[0]
# temp_list2


In [10]:
# data1 = [stat['data'] for stat in response['contents']['stat'] if stat['updnLine'] == 1]
# data2 = [stat['data'] for stat in response['contents']['stat'] if stat['updnLine'] == 0]

In [24]:
df = pd.concat(temp_list)
df['updn'] = 1
df_ = pd.concat(temp_list2)
df_['updn'] = 0
df = pd.concat((df,df_))
df

,dow,hh,mm,congestionTrain,updn
0,MON,08,00,0,1
1,MON,08,10,23,1
2,MON,08,20,46,1
3,MON,08,30,46,1
4,MON,08,40,23,1
...,...,...,...,...,...
1,MON,08,10,191,0
2,MON,08,20,191,0
3,MON,08,30,199,0
4,MON,08,40,199,0


In [26]:
# 최대혼잡도
df_res = df[df['congestionTrain'] != 0].copy()
df_res.drop(columns=['dow','hh','mm'],inplace=True)
df_res = df.groupby('updn')['congestionTrain'].agg('max').reset_index().round(2)
df_res

,updn,congestionTrain
0,0,201
1,1,72
